In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import tensorflow as tf


## Reading the data

In [3]:
data = pd.read_csv(r"..\..\Data\small_ohe.csv")
data.head()

,age,default,housing,loan,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,0.171429,1,-1,1,0.029412,1.0,0.0,0,0.333333,0.269680,...,0,1,0,0,0,1,0,0,0,0
1,0.300000,1,1,1,0.088235,1.0,0.0,0,0.937500,0.698753,...,0,1,0,0,0,1,0,0,0,0
2,0.100000,1,-1,1,0.000000,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,0,0,0,0,1
3,0.285714,1,0,0,0.058824,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,1,0,0,0,0
4,0.414286,1,-1,1,0.000000,1.0,0.0,0,0.687500,0.389322,...,0,0,1,0,0,0,1,0,0,0


In [4]:
data.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'pdays2', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_w

In [5]:
# Saperating features and result vectors
X = data.drop('y', axis=1).values
y = data['y'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [7]:
# Define no sequence of layers
input_tensor = tf.keras.Input(shape=(55,))
layer1 = tf.keras.layers.Dense(64,name = 'layer_1', activation='relu',use_bias=True)(input_tensor)
layer2 = tf.keras.layers.Dense(64,name = 'layer_2', activation='relu',use_bias=True)(layer1)
layer3 = tf.keras.layers.Dense(32,name = 'layer_3', activation='relu',use_bias=True)(layer2)
layer4 = tf.keras.layers.Dense(1, name = 'logit', activation=None,use_bias=True)(layer3)
output_tensor = tf.keras.layers.Activation('sigmoid', name='output')(layer4)
model = tf.keras.Model(inputs=input_tensor, outputs=[layer1,layer2,layer3,layer4,output_tensor]) 
losses = {'logit': 'BinaryCrossentropy'}
model.compile(loss=losses, optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=128, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200
40/40 [==============================] - 1s 5ms/step - loss: 0.7678 - logit_loss: 0.7678 - layer_1_accuracy: 0.0059 - layer_2_accuracy: 0.0668 - layer_3_accuracy: 0.0310 - logit_accuracy: 0.6069 - output_accuracy: 0.5003 - val_loss: 0.5879 - val_logit_loss: 0.5879 - val_layer_1_accuracy: 0.0078 - val_layer_2_accuracy: 0.1154 - val_layer_3_accuracy: 0.0339 - val_logit_accuracy: 0.7087 - val_output_accuracy: 0.4961
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 0.5858 - logit_loss: 0.5858 - layer_1_accuracy: 0.0081 - layer_2_accuracy: 0.1166 - layer_3_accuracy: 0.0320 - logit_accuracy: 0.7177 - output_accuracy: 0.4968 - val_loss: 0.5690 - val_logit_loss: 0.5690 - val_layer_1_accuracy: 0.0124 - val_layer_2_accuracy: 0.1466 - val_layer_3_accuracy: 0.0325 - val_logit_accuracy: 0.7265 - val_output_accuracy: 0.4993
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 0.5556 - logit_loss: 0.5556 - layer_1_accuracy: 0.0088 - layer_2_accu

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 55)]              0         
                                                                 
 layer_1 (Dense)             (None, 64)                3584      
                                                                 
 layer_2 (Dense)             (None, 64)                4160      
                                                                 
 layer_3 (Dense)             (None, 32)                2080      
                                                                 
 logit (Dense)               (None, 1)                 33        
                                                                 
 output (Activation)         (None, 1)                 0         
                                                                 
Total params: 9,857
Trainable params: 9,857
Non-trainable par

## Convert NN to ONNX file

In [10]:
import onnxmltools
# convert the model to ONNX format
onnx_net = onnxmltools.convert_keras(model)
onnxmltools.utils.save_model(onnx_net, "my_model.onnx")
content = onnx_net.SerializeToString()

## Setup onnx

In [26]:
import onnxruntime as ort
from sklearn.neighbors import NearestNeighbors
def setup(onnx_file: str,):
    # Load the ONNX model
    ort_sess = ort.InferenceSession(onnx_file)
    ortvalue = ort.OrtValue.ortvalue_from_numpy(X_train.astype(np.float32), 'cpu')
    layer_output = ort_sess.run(['layer_1','layer_2','layer_3','logit'], {ort_sess.get_inputs()[0].name: ortvalue}) 
    neigh = []
    for i in range(len(layer_output)):
        neigh.append(NearestNeighbors().fit(layer_output[i]))
    return  ort_sess, neigh

ort_sess, neigh = setup('my_model.onnx')

In [21]:
def confidences(x, ort_sess, layer_out, y_train):
    ortvalue = ort.OrtValue.ortvalue_from_numpy(x.astype(np.float32), 'cpu')
    layer_output = ort_sess.run(['layer_1','layer_2','layer_3','logit'], {ort_sess.get_inputs()[0].name: ortvalue}) 
    for i in range(len(x)):
        layer_predict = []
        for j in range(len(layer_output)):
            layer_predict.append(neigh[j].kneighbors(layer_output[j][i].reshape(1, -1), 1, return_distance=False))
    return layer_predict

confidences(X_test, ort_sess, layer_out, y_train)

[array([[0.        , 0.6527649 , 0.        , ..., 0.        , 0.226087  ,
         0.        ],
        [0.        , 0.44003183, 0.        , ..., 0.20403242, 0.3326547 ,
         0.03314645],
        [0.8234054 , 0.42975003, 0.28593206, ..., 0.        , 0.5727266 ,
         0.4528196 ],
        ...,
        [0.        , 0.7380428 , 0.        , ..., 0.50017697, 1.052403  ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.81401134,
         0.47289377],
        [0.        , 0.67658085, 0.        , ..., 0.5704702 , 0.28243923,
         0.        ]], dtype=float32),
 array([[0.93117565, 0.        , 0.        , ..., 0.09432337, 0.61572105,
         0.6949224 ],
        [0.6982644 , 0.        , 0.10560359, ..., 0.        , 0.6548017 ,
         0.9350475 ],
        [0.06099963, 0.7337561 , 0.        , ..., 0.        , 0.26466605,
         0.9146138 ],
        ...,
        [0.04251009, 0.        , 0.        , ..., 0.        , 0.52163696,
         0.67867094